This iPython notebook is to experiment on Lexical features and Tf-Idf vvectorizer

In [1]:
import sys
import os
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import scipy.spatial.distance as ds_metric
from textblob import TextBlob, Word
import gensim 
import io
from collections import defaultdict

/home/surajr/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
with io.open('sarcasm_dataset.txt','r') as fname:
    file_content = fname.readlines()

In [11]:
"""
Function to extract the below features.
1. Presence of emoji's
2. Count of number of Question marks
3. Count of number of Exclamation marks
4. presence of hashtags other than #sarcasm
5. presence of any @user tweets

"""
def lexicalFeatures(tweet_content):
    emoji = {
    "&lt;3" : "positive",":D" : "positive",	":d" : "positive", ":dd" : "positive", ":P" : "positive", ":p" : "positive","8)" : "positive",
    "8-)" : "positive",  ":-)" : "positive",    ":)" : "positive",    ";)" : "positive",    "(-:" : "positive",    "(:" : "positive",
    ":')" : "positive",    "xD" : "positive",    "XD" : "positive",  "yay!" : "positive",  "yay" : "positive",  "yaay" : "positive",
    "yaaay" : "positive",  "yaaaay" : "positive", "yaaaaay" : "positive", "Yay!" : "positive", "Yay" : "positive", "Yaay" : "positive",
    "Yaaay" : "positive", "Yaaaay" : "positive", "Yaaaaay" : "positive",  ":/" : "negative", "&gt;" : "negative", ":'(" : "negative",
    ":-(" : "negative", ":(" : "negative", ":s" : "negative",":-s" : "negative","-_-" : "negative", "-.-" : "negative" }
    lex_feature, lex_label, emoji_count = [], [],defaultdict(int)
    for tweet_content in file_content:
        isPresent, temp = False, []
        tweet, label = tweet_content[:-2].split(), tweet_content[-2]        
        
        for word in tweet:
            if word in emoji:            
                isPresent = True   

        temp.append(isPresent)
        temp.append(int(tweet.count("?")))
        temp.append(int(tweet.count("!")))
        
        other_hashtags=[j[1:] for j in tweet if j.startswith("#")]
        temp.append(1.0 if len(other_hashtags)!=0 else 0.0)
        
        lex_label.append(label)
        lex_feature.append(temp)
    return lex_feature, lex_label 
    


In [12]:
feat, lex_label = lexicalFeatures(file_content)

In [17]:
feat = pd.DataFrame(feat)

In [57]:
encoding="utf-8"
sentences = []
for tweet_content in file_content:
    tweet, label = tweet_content[:-2].split(), tweet_content[-2]
    sentences.append(tweet)
# train word2vec on the tweets
model = gensim.models.Word2Vec(sentences, min_count=3)
model.save('gensim')

Lets add the features to TF-IDF and train the model

In [18]:
import numpy
import pandas
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import sklearn.linear_model as lm
from nltk.corpus import stopwords
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import pipeline,metrics, grid_search 
from numpy import genfromtxt
from sklearn import cross_validation, tree, linear_model
import sklearn.ensemble as ek
from sklearn import cross_validation, tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
import sklearn.linear_model as lm

In [2]:
df = pandas.DataFrame()
df = pandas.read_csv("feature_dataset.csv", header=0)
df.head()

,Unnamed: 0,label,Blob Polarity,Blob Subjectivity,Capitalization,Negative Sentiment,POS_1,POS_2,POS_3,POS_4,...,first half Blob Subjectivity,first half sentiment,negative Sentiment first half,negative Sentiment second half,positive Sentiment first half,positive Sentiment second half,second half Blob Polarity,second half Blob Subjectivity,second half sentiment,sentiment
0,0,1,0.50000,0.6,0,0.031250,2.0,0.0,2.0,0.0,...,0.6,0.593750,0.031250,0.000000,0.625000,0.0000,0.0,0.0,0.000000,0.593750
1,1,1,-0.80000,0.9,0,1.444444,3.0,0.0,3.0,1.0,...,0.9,-0.638889,0.819444,0.625000,0.180556,0.1250,0.0,0.0,-0.500000,-1.138889
2,2,0,-0.50000,1.0,0,0.725000,3.0,1.0,2.0,2.0,...,1.0,-0.600000,0.725000,0.000000,0.125000,0.0000,0.0,0.0,0.000000,-0.600000
3,3,0,-0.80000,0.9,1,0.778409,1.0,0.0,7.0,3.0,...,0.9,-0.541667,0.625000,0.153409,0.083333,0.0625,0.0,0.0,-0.090909,-0.632576
4,4,1,0.78125,0.6,1,0.039062,4.0,0.0,2.0,1.0,...,0.6,0.601562,0.039062,0.000000,0.640625,0.0000,0.0,0.0,0.000000,0.601562


In [4]:
X = df.drop(['label'],axis=1).values
Y = df['label']

In [7]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y ,test_size=0.2)

In [15]:
X_train = pandas.DataFrame(X_train)
X_train = X_train.fillna(X_train.mean())

X_test = pandas.DataFrame(X_test)
X_test = X_test.fillna(X_test.mean())

In [10]:
tfidf_vec=TfidfVectorizer(analyzer="word",max_features=None,strip_accents='unicode',token_pattern=r'\w{1,}',lowercase=True,ngram_range=(1,3),min_df=2,use_idf=True,smooth_idf=True,norm="l2",sublinear_tf=True)

In [19]:
model = { "DecisionTree":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest":ek.RandomForestClassifier(n_estimators=50),
         "Adaboost":ek.AdaBoostClassifier(n_estimators=50),
         "GradientBoosting":ek.GradientBoostingClassifier(n_estimators=50),
         "GNB":GaussianNB(),
         "LogisticRegression":lm.LogisticRegression(C=0.30,penalty="l1")   
}

In [20]:
results_algo = {}
for algo in model:
    clf = model[algo]
    clf.fit(X_train,y_train.astype(int))
    score = clf.score(X_test,y_test.astype(int))
    print ("%s : %s " %(algo, score))
    results_algo[algo] = score

LogisticRegression : 0.771929824561 
RandomForest : 0.744360902256 
GradientBoosting : 0.749373433584 
GNB : 0.706766917293 
DecisionTree : 0.676691729323 
Adaboost : 0.729323308271 
